In [1]:
import numpy as np
import cv2 # OpenCV
import matplotlib
import matplotlib.pyplot as plt
import collections
import os

# iscrtavanje slika u notebook-u
%matplotlib inline
# prikaz vecih slika
matplotlib.rcParams['figure.figsize'] = 16,12

In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            images.append(img)
    return images 

In [4]:
#detekcija knjiga
def load_sets():
    sa_knjigom = load_images_from_folder("skup_podataka/saKnjigom")
    bez_knjige = load_images_from_folder("skup_podataka/bezKnjiga")
    test_skup = load_images_from_folder("skup_podataka/test")
    return sa_knjigom, bez_knjige, test_skup
    
def hog_descriptior(sa_knjigom, bez_knjige, hog):
    pos_features = []
    neg_features = []
    labels = []

    print("-- hog pos imgs")
    br = 1
    for img in sa_knjigom:
        pos_features.append(hog.compute(img))
        labels.append(1)
        if br % 100 == 0 or br > 207:
            print("--- pos", br)
        br = br + 1
    print("-- hog neg imgs")
    br = 1
    for img in bez_knjige:
        neg_features.append(hog.compute(img))
        labels.append(0)
        if br % 100 == 0 or br > 207:
            print("--- neg", br)
        br = br + 1
    print("-- features")
    pos_features = np.array(pos_features)
    neg_features = np.array(neg_features)
    np.vstack((pos_features, neg_features))
    np.array(labels)
    print("-- kraj")
    return
    

def classify_window(window, hog):
    features = hog.compute(window).reshape(1, -1)
    return clf_svm.predict_proba(features)[0][1]

def process_image(image, step_size, hog, window_size=(100, 40)):
    best_score = 0
    best_window = None
    for y in range(0, image.shape[0], step_size):
        for x in range(0, image.shape[1], step_size):
            this_window = (y, x) # zbog formata rezultata
            window = image[y:y+window_size[1], x:x+window_size[0]]
            if window.shape == (window_size[1], window_size[0]):
                score = classify_window(window, hog)
                if score > best_score:
                    best_score = score
                    best_window = this_window
    return best_score, best_window

In [ ]:
# glavna funkcija
with_books, without_books, test_set = load_sets()
print("Slike sa knjigama:", len(with_books))
print("Slike bez knjiga:", len(without_books))
print("Slike za test:", len(test_set))

print("- hog poceo")
nbins = 5 # broj binova
cell_size = (10, 10) # broj piksela po celiji
block_size = (5, 5) # broj celija po bloku

img = with_books[0]

hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1], 
                                  img.shape[0] // cell_size[0] * cell_size[0]),
                        _blockSize=(block_size[1] * cell_size[1],
                                    block_size[0] * cell_size[0]),
                        _blockStride=(cell_size[1], cell_size[0]),
                        _cellSize=(cell_size[1], cell_size[0]),
                        _nbins=nbins)

hog_descriptior(with_books, without_books, hog)
print("- hog prosao")

br = 1
for picture in test_set:
    score, score_window = process_image(image = picture, step_size=10, hog =hog)
    print("slika", br, "dobijena vr", score)
    br = br + 1

Slike sa knjigama: 210
Slike bez knjiga: 210
Slike za test: 150
- hog poceo
-- hog pos imgs
--- pos 100
--- pos 200
--- pos 208
--- pos 209
--- pos 210
-- hog neg imgs
--- neg 100
--- neg 200
--- neg 208
--- neg 209
--- neg 210
-- features
